In [ ]:
from autumn.core.project import get_project, load_timeseries
from matplotlib import pyplot
from autumn.core.plots.utils import REF_DATE
import pandas as pd
from autumn.core.utils.display import pretty_print
import os
from summer.utils import ref_times_to_dti

In [ ]:
project = get_project("sm_sir", "malaysia")

In [ ]:
custom_params = project.param_set.baseline#.update(dict(contact_rate = 0.1))
model = project.run_baseline_model(custom_params)
derived_df = model.get_derived_outputs_df()

In [ ]:
output = ["notifications","infection_deaths"]

all_targets = load_timeseries(os.path.join(project.get_path(), "timeseries.json"))
for target in all_targets:
    all_targets[target].index = ref_times_to_dti(REF_DATE, all_targets[target].index)  
    
notif_values = all_targets[output[0]]
deaths_values = all_targets[output[1]]

notif_times = all_targets[output[0]].index
death_times = all_targets[output[1]].index

#icu_dates, icu_values = get_target_series(project.calibration.targets, REF_DATE, output[2])

In [ ]:
fig = pyplot.figure(figsize=(16, 5))
pyplot.style.use("ggplot")
axis = fig.add_subplot(1,2,1)
axis = derived_df[output[0]].plot()
axis.scatter(notif_times,notif_values, c="k")
pyplot.title("COVID notifications")
print("notifications maximum: "+f"{max(derived_df[output[0]])}")
axis = fig.add_subplot(1,2,2)
axis = derived_df[output[1]].plot()
axis.scatter(death_times, deaths_values, c="k")
pyplot.title("COVID deaths")
print("deaths maximum: "+f"{max(derived_df[output[1]])}")
# axis = fig.add_subplot(1,3,3)
# axis = derived_df[output[2]].plot()
# axis.scatter(icu_dates, icu_values, c="k")
# pyplot.title("icu occupancy")
# print("ICU maximum: "+f"{max(derived_df[output[2]])}")
# fig.show()

In [ ]:
fig = pyplot.figure(figsize=(12, 8))
pyplot.style.use("ggplot")
axis = fig.add_subplot()
axis = derived_df["hospital_occupancy"].plot()

In [ ]:
fig = pyplot.figure(figsize=(15, 6))
pyplot.style.use("ggplot")
axis = fig.add_subplot(1,2,1)
axis = derived_df["cdr"].plot()
axis = fig.add_subplot(1,2,2)
axis = derived_df["prop_ever_infected"].plot()

In [ ]:
fig = pyplot.figure(figsize=(12, 8))
pyplot.style.use("ggplot")
axis = fig.add_subplot()
axis = derived_df["proportion_seropositive"].plot()